```
Summary
This time, I mainly use the python VADER library in the nlp domain to judge the emotions of the article. Through the emotional trend of the monthly latitude and the frequency of uploading, it is found that the author is generally an optimistic and diligent person.
```


In [1]:
!pip install pandas 
!pip install numpy
!pip install seaborn
!pip install matplotlib

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

In [3]:
data = pd.read_csv('data/posts.csv')

In [4]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [5]:
data.head()

,year,month,day,author,title,Filename,Content
0,2001,9,14,ritholtz,How This Page “Happened”,_2001_09_how-this-page-happened_.txt,"""The short version is I wanted to get somethin..."
1,2001,9,16,ritholtz,Postscript,_2001_09_postscript_.txt,"""Postscript\n\nA few small details to tidy up...."
2,2001,9,19,ritholtz,Feedback,_2001_09_feedback_.txt,"""Feedback\n\nEver since I posted the original ..."
3,2001,10,26,ritholtz,Favorite Links,_2001_10_favorite-links_.txt,"""Home\n\nWritings\n\n\n\n\nPublished Articles\..."
4,2001,10,26,ritholtz,Recent Articles,_2001_10_recent-articles_.txt,"""\u201cVolatile markets call for stop-loss ord..."


https://github.com/cjhutto/vaderSentiment#features-and-updates

VADER Sentiment Analysis. VADER (Valence Aware Dictionary and sEntiment Reasoner) is a lexicon and rule-based sentiment analysis tool that is specifically attuned to sentiments expressed in social media, and works well on texts from other domains.

```
positive sentiment: compound score >= 0.05
neutral sentiment: (compound score > -0.05) and (compound score < 0.05)
negative sentiment: compound score <= -0.05
```

In [6]:
!pip install vaderSentiment

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [8]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [9]:
analyzer = SentimentIntensityAnalyzer()

In [10]:
vs = analyzer.polarity_scores(data['Content'].tolist()[0])
print("{}".format(str(vs)))

{'neg': 0.088, 'neu': 0.783, 'pos': 0.129, 'compound': 0.984}


In [ ]:
data['SentimentAll'] = data['Content'].map(lambda s: str(analyzer.polarity_scores(s)))

In [ ]:
data.head()

In [ ]:
data['SentimentAll'].tolist()[0]

In [ ]:
def completion_date(elem):
    """Completion date"""
    string = str(elem)
    if len(string) == 1:
        return "0"+string
    else:
        return string

In [ ]:
data['year'] = data['year'].map(str)
data['month'] = data['month'].map(completion_date)
data['day'] = data['day'].map(completion_date)

In [ ]:
data['date'] = data['year']+data['month']+data['day']

In [ ]:
data.drop(['year', 'month', 'day'], axis=1, inplace=True)

In [ ]:
data['date'] = pd.to_datetime(data['date'])

In [ ]:
def get_sentiment(sentiment):
    """sentiment"""
    sentiment = eval(sentiment)
    if sentiment['compound'] >= 0.05:
        return "pos"
    elif sentiment['compound'] < 0.05 and sentiment['compound']>=-0.05:
        return "neu"
    else:
        return "neg"

In [ ]:
data['Sentiment'] = data['SentimentAll'].map(get_sentiment)

In [ ]:
data['Sentiment'].value_counts()

In [ ]:
sentiment = {
    'pos': 1,
    'neg': -1,
    'neu': 0
}
data['Sentiment'] = data['Sentiment'].map(sentiment)

In [ ]:
data = data.set_index(data['date'])

In [ ]:
data_tmp = data.copy()

In [ ]:
# emotional change graph over time
data_tmp.resample("M")['Sentiment'].mean().plot(figsize=(8, 6))

In [ ]:
# upload time frequency (monthly latitude)
data_tmp.resample("M")['Sentiment'].count().plot(figsize=(8, 6))